# Image Retrieval

In [1]:
%matplotlib inline
%matplotlib notebook
%load_ext autoreload
%autoreload 2
from download import main
from glob import glob
import matplotlib.pyplot as plt
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import os.path as osp

from utils import *
import src as sc

In [2]:
data_path = osp.join('..','data')
agument_path = osp.join(data_path, 'agument')
categories = ['boy', 'cat', 'car', 'dog', 'girl']
interested_frames = ['start', 'middle', 'quater3']
limit_each_category = 1000
vocab_size = 8000
vocab_prefix = 'vocab.pkl'

# load image paths
category_paths = get_image_directories(data_path, categories)
agument_category_paths = get_image_directories(agument_path, categories)
image_paths, image_labels, image_ids = load_images_paths(limit_each_category, category_paths)
agument_paths, agument_labels, agument_ids = load_agument_image_paths(agument_path, image_paths, [2, 1.5])

image_paths = np.concatenate((image_paths, agument_paths), axis=0)
image_labels = np.concatenate((image_labels, agument_labels), axis=0)
image_ids = np.concatenate((image_ids, agument_ids), axis=0)
# X_train, X_test, y_train, y_test = train_test_split(image_paths, image_ids, test_size=0, random_state=42)
X_train, y_train = shuffle(image_paths, image_ids, random_state=0)

## build visual vocabulary

In [3]:
vocab_filename = str(vocab_size) + vocab_prefix
if not osp.exists(vocab_filename):
    vocab = sc.build_vocabulary(X_train, vocab_size)
    with open(vocab_filename, 'wb') as f:
      pickle.dump(vocab, f)
      print('{:s} saved'.format(vocab_filename))

In [ ]:
if not osp.exists('train_feats.pkl'):
    train_feats = sc.get_bags_of_sifts(X_train, vocab_filename)
#     test_feats = sc.get_bags_of_sifts(X_test, vocab_filename)
else:
    with open('train_feats.pkl', 'rb') as f:
        train_feats = pickle.load(f)
#     with open('test_feats.pkl', 'rb') as f:
#         test_feats = pickle.load(f)

## Weighting Features with tf-idf

In [ ]:
train_feats_weighted = sc.tf_idf(train_feats)

In [ ]:
predicted_indexes = sc.nearest_neighbor_classify(train_feats_weighted, train_feats_weighted)

In [ ]:
acc = np.array(y_train)[predicted_indexes[:, :3].T] == np.array(y_train)
acc = np.mean(acc.astype(int), axis=1)
print(np.sum(acc))

In [ ]:
# test_feats = sc.get_bags_of_sifts(['/Users/pagepeng/Desktop/test.png'], vocab_filename)

In [ ]:
# predicted_indexes = sc.nearest_neighbor_classify(train_feats, test_feats)

## display the rankings of matching
rank = (sum(Ri)-(Nrel\*(Nrel+1)/2)/(N\*Nrel)

In [ ]:
ranks = sc.rank(predicted_indexes, y_train, 3)
print(np.average(ranks))

In [ ]:
print(y_train[predicted_indexes[0, :10]])

## display some predictions

In [ ]:
similar_image_num = 10
for i in range(10, 40, 2):
    print(X_train[i])
    plt.subplots(1,similar_image_num+1, figsize=(20,20))
    im = load_gif_gray(X_train[i])
    plt.subplot(1,similar_image_num+1,1)
    plt.imshow(im, cmap='gray')
    for j in range(similar_image_num):
        im = load_gif_gray(X_train[predicted_indexes[i, j]])
        plt.subplot(1,similar_image_num+1,j+2)
        plt.imshow(im, cmap='gray')

# Classification with linear SVM

In [ ]:
print('Using SVM classifier to predict test set categories')
predicted_categories, svms = sc.svm_classify(train_feats, y_train, train_feats, SVM_lambda=1000)

In [ ]:
show_results(X_train, X_train, y_train, y_train, categories, categories,
             predicted_categories)

In [ ]:
weights = np.zeros((1, 600))
for category, svm in svms.items():
    print(svm.coef_)